# Introduction

QCD is, of course, the theory of quarks and gluons.  At high energies the precision with which it can describe mesons and their properties is essential for understanding experimental signals, such as collider signals at the LHC.

In contrast, the low-energy sector of QCD---the sector of protons, neutrons, and their interactions---is much more poorly understood.  Or, rather, the precision with which we can extract numbers about nuclear physics from QCD remains much more limited---there still exist theories of nuclear physics that are not quantitatively grounded in the Standard Model.

In the last decade or so the single-nucleon sector has come under excellent control.  The nucleon masses and the proton-neutron mass splitting have been determined at the physical point, in the continuum limit, for example.  A steady march towards precision single-nucleon matrix elements is under way.

Once you leave the single-nucleon sector, however, the field is much less developed.  However, multinucleon physics is interesting for a variety of experimental programs, such as low-energy underground BSM and neutrino experiments.  The targets in these experiments are not single nucleons but rather atomic nuclei.  So to interpret any signal from these experiments as constraints on new physics requires disentangling all the effects of many-body nuclear physics and QCD to pull out whatever may be new.

QCD is also, obviously, the foundation on top of which nuclear physics, in principle, is built.  However, making this connection quantitative has remained an outstanding problem since it became clear QCD was, in fact, the theory of the strong nuclear force.

The simplest multi-nucleon sector is the two nucleon sector.  Here, the simplest observables include the binding energies of any bound states and the scattering phase shifts.  We still lack a determination of these quantities from QCD at physical pion masses; at heavier pion masses there are still no calculations in the continuum limit.

Scattering is a real-time process.  It also formally relies on asymptotically separated states.  However, on the lattice, typically we have neither: we usually work in Euclidean time to have a well-defined probability measure and we work in a finite volume so that our calculations require a finite amount of RAM and a finite amount of execution time.  There is a no-go theorem due to Maiani and Testa that seemingly prevented access to scattering observables.

However, Lüscher taught us how to turn these seeming limitations to our advantage.  He provided a map from the finite volume spectrum (which can be determined in Euclidean time---or real time, if you happened to have a method to do so!) to the infinite-volume phase shifts at those energies that appear in that finite volume.  Then, by changing the volume we work in we can change the allowed energies and fill in the phase shifts as a function of momentum.

In this notebook we will work through a simple problem in the same spirit: two interacting nonrelativistic particles in one spatial dimension.  

The approach will be as follows:
1. We will set up the problem and study its infinite-volume case.
2. We will set up the same system but where the two particles on a ring of radius $L$.  We will derive the quantization condition.  The quantization condition is the map from the finite-volume energies to scattering phase shift.
3. We will discretize the finite-volume problem and extract its spectrum for a variety of radii.
4. Using the quantization condition, translate the energies found into phase shifts and compare with what we found in the infinite-volume case.

In lattice QCD we do not have access to the nuclear Hamiltonian.  However, we can nevertheless determine its spectrum and use the quantization condition to translate those results into phase shifts!

# One-Dimensional Quantum Mechanics Setup

Consider two quantum mechanical particles that move in one infinite spatial dimension.  Let their interaction potential $V$ depend only on their separation.  Then, their Hamiltonian is given by
$$ \mathcal{H} = \frac{p_1^2}{2 m_1} + \frac{p_2^2}{2 m_2} + V(x_1-x_2) $$

We can switch to center-of-mass and relative coordinates.  Let
\begin{align}
    X &= \frac{m_1 x_1 + m_2 x_2}{M}    &    x &= x_1 - x_2          \\
    P &= p_1+p_2                        &    p &= \frac{m_2 p_1 - m_1 p_2}{M}                \\
    M &= m_1 + m_2                      &  \mu &= \frac{m_1 m_2}{M}
\end{align}
Then it is easy to check the canonical commutation relations $[X,P]=[x,p]=i$ hold, and that $[X,p]=[x,P]=0$, so that we have completely decoupled the center-of-mass and relative motion.

The Hamiltonian may be rewritten in the new variables,
\begin{equation}
    \mathcal{H} = \frac{P^2}{2 M} + \frac{p^2}{2 \mu} + V(x)
\end{equation}

In these coordinates, it's clear that, as long as $V$ is even, there is a parity symmetry,
\begin{align}
    x &\rightarrow -x      &
    p &\rightarrow -p.
\end{align}
In the parity-odd sector the wavefunction will be zero at $x=0$ and won't feel the interaction at all, but we should expect to find the parity-even sector to feel the interaction.

Moreover, the center of mass is completely free.  So, the physical observables can only be a function of the relative momentum $p$.

# Infinite Volume Scattering

# Finite-Volume Quantization Condition

Now consider the same Hamiltonian where each particle is on the same one-dimensional ring of radius $L$.  That is, we identify $-\pi L$ with $\pi L$.  The periodicity is easily expressed in terms of the variables $x_1$ and $x_2$:
\begin{align}
    \psi(x_1 + 2 \pi L, x_2) &= \psi(x_1, x_2) \\
    \psi(x_1, x_2 + 2 \pi L) &= \psi(x_1, x_2) \\
\end{align}
Notice that here we could include *twisted boundary conditions*, where each particle can pick up a phase over the course of an orbit.  In QCD we twist quark fields and the hadrons' twists are automatically determined.

We need to change this restriction on the wavefunction into a quantization condition that relates the energies and phase shifts.

# Discretization

## Formalism

We will get the spectrum of the finite-volume Hamiltonian by direct diagonalization.  In contrast, the finite-volume spectrum in QCD is extracted by measuring correlation functions.  The Lüscher formalism doesn't care how you obtained the finite-volume spectrum---it simply tells you how to take that spectrum and extract infinite-volume phase shifts.

We'll slowly build up the Hamiltonian $\mathcal{H}$.  We'll think of the wavefunction $\psi$ as the vectors on which operators work.  Each entry in the vector will represent the wavefunction at a different spacetime point.

We just need simple linear algebra and plotting utilities:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Lattice

We need to discretize a ring with radius $L$.  One might want all sorts of meshes.  Here we will just consider a mesh with uniform spacing.  Let us conventionally discretize the range $L * [-\pi,\pi)$ with $N$ evenly-spaced points.

In [ ]:
def uniform_discretization(N, L):
    return np.pi * L * np.arange(-1,1,2/N)

Let's visualize progressively finer uniform discretizations for a few radii.

In [ ]:
mesh_refinement = plt.figure()
axes = mesh_refinement.add_subplot(1,1,1)
axes.set_aspect('equal')

Ls = [1,2]
Ns = 2**np.arange(2,6)
for L in Ls:
    for N in Ns:
        ring = uniform_discretization(N,L)
        axes.plot(L*np.cos(ring/L), L*np.sin(ring/L), label="N=%i L=%i"%(N,L))

plt.legend(bbox_to_anchor=(1.04,1))
plt.show()

So, at larger radii we should maybe expect worse discretization effects for a fixed $N$.

Also, note what happens if we make $N$ odd:

In [ ]:
odd_n = plt.figure()
axes = odd_n.add_subplot(1,1,1)
axes.set_aspect('equal')


L=1
for N in 2*np.arange(3,10,2)+1:
    ring = uniform_discretization(N,L)
    axes.plot(L*np.cos(ring/L), L*np.sin(ring/L), label="N=%i L=%i"%(N,L))

plt.show()

We *always* miss $x=0$.  So if we're going to put some sort of regularized Dirac delta function potential down, we'd better be careful!

## Plotting an operator's spectrum and eigenfunctions

In [ ]:
def plot_spectrum(axis, lattice, eigenvalues, eigenvectors, modes=None):
    
    vals = eigenvalues
    vecs = eigenvectors
    if modes is not None:
        vals = vals[0:modes]
        vecs = vecs[0:modes]

    for v,f in zip(vals,vecs):
        wavefunction, = axis.plot(lattice,abs(v)*(np.sign(v)+f), linestyle='-')
        axis.hlines(v,min(lattice),max(lattice), color = wavefunction.get_color(), linestyle = ":")

    return None

## Kinetic Energy

Since we're only considering uniform meshes we can easily write the second-derivative operator as a tri-diagonal matrix, using the finite-difference approximation
\begin{equation}
  f''(x_i) = \frac{f(x_{i+1})-2 f(x_i) + f(x_{i-1})}{(\Delta x)^2} + \mathcal{O}(\Delta x)
\end{equation}

The kinetic energy operator for a particle with mass $m$ is given by
\begin{equation}
    T_m = - \frac{1}{2 m} \partial^2
\end{equation}
We need to specify both a mass and a discretization.

The function `T` is of type `mass --> discretization --> operator`.  It currently assumes a uniform discretization and periodic boundary conditions.

In [ ]:
def T(mass):
    minus_one_over_twice_mass = -1./(2*mass)

    def curried(discretization):
        spatial_dimension=len(discretization)
        operator = np.zeros([spatial_dimension,spatial_dimension])


        # The below assumes a uniform discretization!
        dx = discretization[1] - discretization[0]
        one_over_dx_squared = 1/(dx**2)

        factor = one_over_dx_squared * minus_one_over_twice_mass

        for i in range(spatial_dimension):
            operator[i,i] = -2. * factor
        for i in range(1,spatial_dimension):
            operator[i,i-1] = +1. * factor
        for i in range(spatial_dimension-1):
            operator[i,i+1] = +1. * factor

        # Periodic boundary conditions:
        operator[0,-1] = +1. * factor
        operator[-1,0] = +1. * factor

        return operator
    return curried

We can visualize the stencil that the kinetic energy operator makes.

In [ ]:
L=1
N=32
lattice = uniform_discretization(N,L)
mass = 1
spacing = 2*np.pi*L/N

KE = T(mass)(lattice)

kinetic_operator = plt.figure()
axes = kinetic_operator.add_subplot(1,1,1)
axes.set_aspect('equal')

axes.imshow(KE)
plt.show()

And we can get the spectrum of the free particle on a ring.

We know the exact solution has wavefunctions like $exp(i n x / L)$ and thus momenta that take values $n/L$.  The finite difference is
\begin{equation}
 \frac{1 - \cos(a p)}{a^2}
\end{equation}
and we can check if our operator gives that dispersion relation.

In [ ]:
momentum = np.arange(N/2)/(L)
energy = (1-np.cos(spacing*momentum))/spacing**2
plt.plot(momentum,energy)

free_particle = np.linalg.eigh(KE)
# Only show the modes with positive momentum:
plt.plot(free_particle[0][::2],"+")
plt.show()

Alternatively, we can plot the spectrum with the wavefunctions:

In [ ]:
figure = plt.figure()
axis = figure.add_subplot(111)

plot_spectrum(axis, lattice, free_particle[0], free_particle[1].T, 10)

We can see that the wavefunctions are losing their smoothness, pretty low in the spectrum.

## Potential Energy

The potential energy is easier than the kinetic, because it is entirely local: doesn't contain derivatives.  It is therefore put on the lattice very simply: we just evaluate the potential on each site and stick that on the diagonal of the operator.

The function `V` is of type `potential --> discretization --> operator`.  Later I will give an example of how to construct the `potential` argument, which is a function from `position --> energy`.

In [ ]:
def V(potential):
    def curried(discretization):
        spatial_dimension=len(discretization)
        operator = np.zeros([spatial_dimension,spatial_dimension])

        for i in range(spatial_dimension):
            operator[i,i] = potential(discretization[i])

        return operator
    return curried

## Hamiltonian

The Hamiltonian $\mathcal{H}$ is simply the sum of the kinetic and potential energies.

It is of type `(T, V) --> discretization --> operator`.

In [ ]:
def hamiltonian(kinetic, potential):
    def curried(discretization):
        return kinetic(discretization) + potential(discretization)
    return curried

# A Specific Problem of Interest: The Dirac $\delta$ Function

Let us now simplify to the case of a contact interaction,
\begin{equation}
    V(x) = g \delta(x)
\end{equation}
positive (negative) $g$ is repulsive (attractive) and $\delta$ is the Dirac delta function.

Let's regulate it as a square potential of width $\epsilon$ and height $1/\epsilon$ centered on zero:

In [ ]:
def dirac_delta(g, epsilon):
    def curried(x):
        if -epsilon <= x <= +epsilon:
            return g / epsilon 
        else:
            return 0
    return curried

In [ ]:
g = -1.23
ep = 0.01

predicted_binding_energy = -g**2 / 2;
wavefunction_decay_scale = np.sqrt(2*np.abs(predicted_binding_energy))
circumference = 2*3*wavefunction_decay_scale

lattice=uniform_discretization(np.floor(circumference/ep), circumference/(2*np.pi))

volume = lattice[-1] - lattice[0]
spacing = lattice[1] - lattice[0]
potential = dirac_delta(g, spacing)

H=hamiltonian(T(mass), V(potential))

eigenvalues, eigenfunctions = np.linalg.eigh(H(lattice))

# Annoyingly one cannot zip over the eigenvalues and vectors as-is:
eigenfunctions = eigenfunctions.T

In [ ]:
plt.plot(eigenvalues,"+")
plt.show()

In [ ]:
interacting_spectrum = plt.figure()

axis = interacting_spectrum.add_subplot(111)

plot_spectrum(axis, lattice, eigenvalues, eigenfunctions, 5)
# axis.plot(lattice, [potential(x) for x in lattice])

plt.show()